LOAD MODELS


In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 154.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
!pip install peft

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct")
claify_coder_model = PeftModel.from_pretrained(base_model, "jie-jw-wu/clarify-coder")

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer_qwen = AutoTokenizer.from_pretrained("Qwen/CodeQwen1.5-7B-Chat", device_map={"": "cpu"})
qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/CodeQwen1.5-7B-Chat")

tokenizer_clarify = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map={"": "cpu"})

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
import google.generativeai as genai
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('gemini-2.5-pro')

QWEN LLM

In [ ]:
# Load model directly
def qwen_model_token_generator(prompt):
  global qwen_model, tokenizer_qwen
  messages = [
      {"role": "user", "content": prompt},
  ]
  inputs = tokenizer_qwen.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
  ).to(qwen_model.device)

  if "token_type_ids" in inputs:
    del inputs["token_type_ids"]


  outputs = qwen_model.generate(**inputs, max_new_tokens=256)
  return tokenizer_qwen.decode(outputs[0][inputs["input_ids"].shape[-1]:])

CLARIFY CODER LLM

In [ ]:
def clarify_model_token_generator(prompt):
  global claify_coder_model, tokenizer_clarify
  messages = [
      {"role": "user", "content": prompt},
  ]
  inputs = tokenizer_clarify.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
  ).to(claify_coder_model.device)

  if "token_type_ids" in inputs:
    del inputs["token_type_ids"]

  outputs = claify_coder_model.generate(**inputs, max_new_tokens=256)
  return tokenizer_clarify.decode(outputs[0][inputs["input_ids"].shape[-1]:])

In [ ]:
qwen_model_token_generator("What model are you")

'I am Qwen, a pre-trained language model developed by Alibaba Cloud. I am designed to answer a wide range of questions and provide helpful responses. How can I assist you today?<|im_end|>'

In [ ]:
clarify_model_token_generator("What model are you")

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


IndexError: index out of range in self

LOAD DATASET

In [ ]:
#HUMANEVAL DATASET
from datasets import load_dataset
human_eval_ds = load_dataset("openai/openai_humaneval")

README.md: 0.00B [00:00, ?B/s]

openai_humaneval/test-00000-of-00001.par(…):   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

In [ ]:
#HUMANEVALCOM DATASET
from datasets import load_dataset

dhuman_evalcom_ds = load_dataset("jie-jw-wu/HumanEvalComm")

README.md: 0.00B [00:00, ?B/s]

HumanEvalComm_v2.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/164 [00:00<?, ? examples/s]

In [ ]:
#data preparation
import pandas as pd

# Convert the dataset to a pandas DataFrame for easier manipulation
df = dhuman_evalcom_ds["train"].to_pandas()

# Define the prompt columns to iterate over
prompt_columns = ['prompt', 'prompt1a', 'prompt1c', 'prompt1p', 'prompt2ac', 'prompt2ap', 'prompt2cp', 'prompt3acp']

# Create an empty list to store the new rows
new_rows = []

# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    # For each row, iterate over the prompt columns
    for prompt_col in prompt_columns:
        # Create a new row with the specified columns

        if prompt_col == 'prompt':
            question_category = "original"
        elif prompt_col == 'prompt1a':
            question_category = "ambigious"
        elif prompt_col == 'prompt1c':
            question_category = "inconsistent"
        elif prompt_col == 'prompt1p':
            question_category = "incomplete"
        elif prompt_col == 'prompt2ac':
            question_category = "ambigious and inconsistent"
        elif prompt_col == 'prompt2ap':
            question_category = "ambigious and incomplete"
        elif prompt_col == 'prompt2cp':
            question_category = "inconsistent and incomplete"
        elif prompt_col == 'prompt3acp':
            question_category = "ambigious, inconsistent, and incomplete"

        if row[prompt_col] == None:
            continue

        new_row = {
            'entry_point': row['entry_point'],
            'name': row['name'],
            'prompt': row[prompt_col],  # Use the current prompt column
            'solution': row['solution'],
            'test_case': row['test_case'],
            'modified_question': False if prompt_col == 'prompt' else True,
            'question_category': question_category
        }
        # Append the new row to the list
        new_rows.append(new_row)

# Create a new DataFrame from the list of new rows
new_df = pd.DataFrame(new_rows)

# Display the first few rows of the new DataFrame(important)
display(new_df.head(3))

,entry_point,name,prompt,solution,test_case,modified_question,question_category
0,has_close_elements,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,from typing import List\n\n\ndef has_close_ele...,"[{'input': '[1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0....",False,original
1,has_close_elements,HumanEval/0,from typing import List\n\n\ndef candidate(num...,from typing import List\n\n\ndef has_close_ele...,"[{'input': '[1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0....",True,ambigious
2,has_close_elements,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,from typing import List\n\n\ndef has_close_ele...,"[{'input': '[1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0....",True,inconsistent


In [ ]:
import re, json

def extract_code_from_output(output: str) -> str:
    """
    Extracts the Python code from a model output that should contain JSON.

    Returns:
        A string containing just the Python code ("" if not found).
    """
    # Use regex to find the JSON object
    json_match = re.search(r'\{.*\}', output, re.DOTALL)
    if json_match:
        json_string = json_match.group(0)
        try:
            parsed = json.loads(json_string)
            if "code" in parsed:
                return parsed["code"]
        except json.JSONDecodeError:
            print(f" Warning: Could not decode JSON: {json_string}")
    else:
        print(f"No JSON object found in model output:\n{output}")

    return ""  # Always return a string, even on failure


In [ ]:
def parse_json_from_output(text: str):
    try:
        match = re.search(r"\{.*?\}", text, re.DOTALL)
        if match:
            obj = json.loads(match.group(0))
            # Validate schema
            if "is_good_question" in obj and "answer" in obj:
                return obj
            else:
                print("⚠️ JSON found but does not match schema")
                return {"is_good_question": False, "answer": ""}
        else:
            print("⚠️ No JSON object found")
            return {"is_good_question": False, "answer": ""}
    except json.JSONDecodeError as e:
        print(f"⚠️ JSON decode error: {e}")
        return {"is_good_question": False, "answer": ""}

In [ ]:
def extract_json(output: str) -> dict:
    json_match = re.search(r'\{.*\}', output, re.DOTALL)
    if json_match:
        json_string = json_match.group(0)
        try:
            return json.loads(json_string)
        except json.JSONDecodeError:
            print(f"Could not decode JSON: {json_string}")
    else:
        print(f"No JSON found in output:\n{output}")
    return {}


In [ ]:
import json
import re

def safe_extract_json(text: str) -> dict:
    """Extract and safely parse JSON from model output."""
    try:
        # Find first {...} block
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if not match:
            print("No JSON found in output:\n", text)
            return {"decision": "NO_QUESTION", "questions": []}
        json_str = match.group(0)

        # Parse JSON
        parsed = json.loads(json_str)

        # Ensure keys exist
        if "decision" not in parsed:
            parsed["decision"] = "NO_QUESTION"
        if parsed["decision"] == "ASK" and "questions" not in parsed:
            parsed["questions"] = []

        return parsed
    except Exception as e:
        print("JSON parse error:", e, "\nRaw output:", text)
        return {"decision": "NO_QUESTION", "questions": []}

ROUND 1 IMPLEMENTATION

In [ ]:
import json
def round_1_code_generation(question):
  prompt_for_round_1 = f"""
  You are a Python coding agent.
  Your ONLY job is to take the user’s problem statement and return valid Python code wrapped in strict JSON.

  Input:
  Problem statement: {question}

  Output format:
  {{
    "code": "<python_code_here>"
  }}

  Rules:
  - "code" must contain only executable Python (no explanations, no comments, no markdown).
  - All line breaks in the code must be represented with \n inside the string.
  - Use 4 spaces for indentation (Python standard).
  - Ensure indentation is preserved exactly as valid Python requires.
  - Do not include extra keys or fields.
  - Do not return raw code outside JSON.
  - Always return a complete, working solution.
  """

  code = qwen_model_token_generator(prompt_for_round_1)
  json_code = extract_code_from_output(code)
  return json_code

ROUND 2 IMPLEMENTATION

In [ ]:
def round_2_implmentation(question, code):
  prompt_for_round_2 = f"""
    You are ClarifyCoder, a clarification-focused coding agent.
    Your job is to decide if the given problem statement and current code are clear enough to implement, or if a clarifying question is required.
    Always return strict JSON.

    Input:
    Problem statement: {question}
    Current code: {code}

    Output format (two possible cases):

    1. If NO clarification is needed:
    {{
      "decision": "NO_QUESTION"
    }}

    2. If clarification IS needed:
    {{
      "decision": "ASK",
      "questions": "<concise, high-value clarifying question here>"
    }}

    Rules:
    - Output ONLY valid JSON, no explanations, no markdown.
    - When asking, keep questions minimal and essential (no more than 2 per turn).
    - Do not restate the problem or code; only output questions.
    - Questions must be concrete and resolve ambiguity, inconsistency, or missing information.
    - Never output both "NO_QUESTION" and "ASK" at the same time.
  """

  raw_output = clarify_model_token_generator(prompt_for_round_2)
  return safe_extract_json(raw_output)

  # decision = clarify_model_token_generator(prompt_for_round_2)
  # json_decision = extract_code_from_output(decision)
  # return extract_json(json_decision)


ROUND 3 IMPLEMENTATION

In [ ]:
def round_3_implmentation(question, code, clarifying_question, answer_to_clarifying_question):
  prompt_for_round_3 = f"""
    You are a Python coding agent.
    You are given:
    - The original problem statement.
    - Clarifying Q/A pairs that resolve ambiguity, inconsistency, or missing requirements.
    - (Optional) Previous code attempts or test feedback.

    Input format:
    Problem statement: {{question}}
    Current code: {{code}}
    Clarifying Question: {{clarifying_question}}
    Answer to Clarifying Question: {{answer_to_clarifying_question}}

    Your task: Generate a corrected and complete Python solution.

    Output format:
    {{
      "code": "<python_code_here>"
    }}

    Rules:
    - "code" must contain ONLY valid, executable Python code.
    - Do not include explanations, reasoning, or comments.
    - Do not include markdown formatting or extra keys.
    - Preserve Python indentation strictly: use 4 spaces.
    - Represent line breaks as \n inside the JSON string.
    - Always return a self-contained, working solution (not just a patch).
  """
  code = qwen_model_token_generator(prompt_for_round_3)
  json_code = extract_code_from_output(code)
  return json_code


LLM EVALUATOR

In [ ]:
import json
import google.generativeai as genai
from google.colab import userdata

def clarifying_question_answer(gemini_model,original_question, clarifying_question, modified_question_category):
  prompt_template = """You are the LLM Evaluator (Answerer).
    You will be given:
    1. The original problem statement (full, unmodified).
    2. A clarifying question about that problem.

    Your tasks:
    - First, determine if the clarifying question is GOOD or BAD:
      - GOOD QUESTION = directly resolves an ambiguity, inconsistency, or missing information in the modified problem.
      - BAD QUESTION = irrelevant, redundant, or does not resolve any ambiguity/missing info.
    - Then, if the question is GOOD, provide a direct answer to it using only the original problem.
    - Do not restate the problem, explain your reasoning, or add hints.
    - Always output strict JSON.

    INPUT:
    Problem statement: {original_question}
    Clarifying question: {clarifying_question}
    Missing Information Category: This is an {modified_question_category} question.

    Output format:
    {{
      "is_good_question": true|false,
      "answer": "<direct_answer_or_empty_string>"
    }}

    Rules:
    - If is_good_question = true → fill "answer" with the direct factual answer.
    - If is_good_question = false → set "answer" to "" (empty string).
    - Do not output anything outside JSON.
  """

  #use gemini API to generate the answer
  # GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
  # genai.configure(api_key=GOOGLE_API_KEY)
  # gemini_model = genai.GenerativeModel('gemini-2.5-pro')

  prompt = prompt_template.format(
        original_question=original_question,
        clarifying_question=clarifying_question,
        modified_question_category=modified_question_category
    )

  response = gemini_model.generate_content(prompt)
  try:
    # Assuming the response is a JSON string
    response_text = response.candidates[0].content.parts[0].text
    return parse_json_from_output(response_text)
  except Exception as e:
    # Handle cases where the response is not valid JSON
    print(f"Warning: Could not decode Gemini response: {e}")
    return {"is_good_question": False, "answer": ""}

AGENTIC FLOW

In [ ]:
def test_generated_code(generated_code, test_case):
    """
    Tests the generated Python code against the provided test case.

    Args:
        generated_code: A string containing the generated Python code.
        test_case: A string containing the test case code with assertions.

    Returns:
        True if the generated code passes the test case, False otherwise.
    """
    try:
        # Create a dictionary to act as the execution environment
        exec_env = {}

        # Execute the generated code
        exec(generated_code, exec_env)

        # Execute the test case in the same environment
        exec(test_case, exec_env)

        # If no exception is raised, the test case passed
        return True
    except AssertionError:
        # If an AssertionError is raised, the test case failed
        return False
    except Exception as e:
        # Handle other potential exceptions during execution
        print(f"An error occurred during code execution: {e}")
        return False

# Example usage (assuming 'final_code' and 'row['test_case']' are available after running the agentic flow)
# for index, row in new_df.iterrows():
#   final_code = run_agentic_flow(row)
#   passed = test_generated_code(final_code, row['test_case'])
#   print(f"Entry {row['entry_point']} test result: {passed}")

In [ ]:
def run_agentic_flow(row):
  question = row["prompt"]
  code = round_1_code_generation(question)
  question_category = row["question_category"]
  initial_code = code
  final_code = None
  number_iterations = 0
  asked_follow_up_question = False
  is_good_question = False

  for i in range(3):  # Maximum 3 rounds of clarification
    decision = round_2_implmentation(question, code)
    print(f"Decision from round_2_implmentation: {decision}") # Print the decision dictionary for inspection
    all_time_answer_to_clarifying_questions = ""
    number_iterations += 1

    if "decision" in decision and decision["decision"] == "NO_QUESTION":
      print(f"Round {i+1}: No clarification needed. Final code generated.")
      # You can add code here to store or process the final code
      break
    elif "decision" in decision and decision["decision"] == "ASK":
      asked_follow_up_question = True
      print(f"Round {i+1}: Clarification needed: {decision['questions']}")
      result_from_clarifying_question = clarifying_question_answer(gemini_model,question, decision["questions"], question_category)
      answer_to_clarifying_question = result_from_clarifying_question["answer"]
      is_good_question = result_from_clarifying_question["is_good_question"]
      all_time_answer_to_clarifying_questions += answer_to_clarifying_question + "\n"
      print(f"Round {i+1}: Answer to clarifying question: {answer_to_clarifying_question}")
      final_code = round_3_implmentation(question, code, decision["questions"], all_time_answer_to_clarifying_questions)
      print(f"Round {i+1}: Code updated.")
    else:
        print(f"Round {i+1}: Unexpected decision format: {decision}. Cannot proceed with clarification.")
        break


  # If the loop finishes without breaking, it means clarification was still needed after 3 rounds or there was an unexpected decision format
  if "decision" not in decision or decision["decision"] != "NO_QUESTION":
      print("Maximum clarification rounds reached or unexpected decision format. Final code might be incomplete.")
  return initial_code, final_code, number_iterations, asked_follow_up_question, is_good_question

In [26]:
total_number_of_questions = new_df.shape[0]
total_number_of_modified_questions = new_df[new_df["modified_question"] == True].shape[0]
pass_at_1_count = 0
test_pass_count = 0
communication_count = 0
good_question_count = 0


for index, row in new_df.iterrows():
  initial_code, final_code, number_iterations, asked_follow_up_question, is_good_question = run_agentic_flow(row)
  pass_at_1_result = test_generated_code(initial_code, row['test_case'])
  test_pass_result = test_generated_code(final_code, row['test_case'])

  if pass_at_1_result:
    pass_at_1_count += 1

  if test_pass_result:
    test_pass_count += 1

  if asked_follow_up_question:
    communication_count += 1
    if is_good_question:
        good_question_count += 1




Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Decision from round_2_implmentation: {'decision': 'ASK', 'questions': "What is the definition of 'closer to each other' in the problem statement?"}
Round 1: Clarification needed: What is the definition of 'closer to each other' in the problem statement?
Round 1: Answer to clarifying question: 


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


No JSON object found in model output:

Problem statement:
You're working with an array of positive integers. Your task is to find the largest product that can be obtained by multiplying two numbers in the array.

Clarifying Question:
How would you implement the missing parts in the below code to find the largest product of two elements in an array?

```python
def max_product(nums):
    """Given an array of positive integers, find the largest product of two numbers."""
    # Your code here
```

Answer to Clarifying Question:
To find the largest product, you need to multiply all possible pairs and return the maximum. Here's how you could solve it:

```python
def max_product(nums):
    """Given an array of positive integers, find the largest product of two numbers."""
    if len(nums) < 2:
        return None  # No two numbers to multiply if there's less than two numbers.
    # Sort the list in ascending order so the smallest two numbers will be the first two.
    nums.sort()
    # The la

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


No JSON object found in model output:
Problem statement:
Create a Python function that takes an integer n and returns a list containing its prime factors in ascending order. The prime factors should be listed repetitively according to their frequency in the prime factorization of n.

Clarifying question:
What is the expected output when calling the function with input 12?

Answer to clarifying question:
For the input 12, the prime factorization is 2^2 * 3^1, so the expected output is [2, 2, 3].

Corrected and complete code:
```python
import math

def prime_factors(n):
    factors = []
    for i in range(2, int(math.sqrt(n)) + 1):
        while n % i == 0:
            factors.append(i)
            n //= i
    if n > 1:
        factors.append(n)
    return factors

result = prime_factors(12)
print(result)
```<|im_end|>
Round 1: Code updated.
Decision from round_2_implmentation: {'decision': 'ASK', 'questions': 'What is the expected behavior when the input list is empty?'}
Round 2: Clarif

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


No JSON object found in model output:


Problem statement:
Write a function called `count_unique_vowels` which takes a single string argument and returns the count of unique vowels present in the string. The vowels are 'a', 'e', 'i', 'o', and 'u' (case-insensitive).

Current code:
```python
def count_unique_vowels(s):
    """
    Count the number of unique vowels in a string.

    :param s: Input string
    :return: Number of unique vowels in the string
    """

    vowels = set('aeiouAEIOU')
    unique_vowels = set()

    for char in s:
        if char in vowels:
            unique_vowels.add(char)

    return len(unique_vowels)
```

Clarifying Question:
Why is the given code incorrect?

Answer to Clarifying Question:
The given code is not correct because it only checks for unique vowels in the input string, but it does not consider the case-insensitivity of the vowels. The code should be modified to
Round 2: Code updated.
Decision from round_2_implmentation: {'decision': 'ASK', 'ques

KeyboardInterrupt: 

In [40]:
print(communication_count)
print(pass_at_1_count)
print(test_pass_count)
print(communication_count)
print(good_question_count)

32
16
30
32
24


In [36]:
# Assume you saved progress earlier
last_index = 32   # <-- replace this with the value you loaded from your saved progress
total_done = last_index + 1   # because index starts at 0

# Now compute totals only for processed rows
total_number_of_questions_new = total_done
total_number_of_modified_questions_new = new_df.loc[:last_index, "modified_question"].sum()

print(f"Total processed questions: {total_number_of_questions_new}")
print(f"Total processed modified questions: {total_number_of_modified_questions_new}")


Total processed questions: 33
Total processed modified questions: 27


In [33]:
print(index)

32


In [31]:
# Save current progress (metrics + last processed index)
progress = {
    "last_index": index,  # <-- index is updated each iteration
    "pass_at_1_count": pass_at_1_count,
    "test_pass_count": test_pass_count,
    "communication_count": communication_count,
    "good_question_count": good_question_count,
}

import json
with open("progress_checkpoint.json", "w") as f:
    json.dump(progress, f)

print("✅ Progress saved:", progress)


✅ Progress saved: {'last_index': 32, 'pass_at_1_count': 16, 'test_pass_count': 30, 'communication_count': 32, 'good_question_count': 24}


In [27]:
print("Current Metrics:")
print(f"pass_at_1_count = {pass_at_1_count}")
print(f"test_pass_count = {test_pass_count}")
print(f"communication_count = {communication_count}")
print(f"good_question_count = {good_question_count}")


Current Metrics:
pass_at_1_count = 16
test_pass_count = 30
communication_count = 32
good_question_count = 24


In [28]:
import json

metrics = {
    "pass_at_1_count": pass_at_1_count,
    "test_pass_count": test_pass_count,
    "communication_count": communication_count,
    "good_question_count": good_question_count
}

with open("metrics_checkpoint.json", "w") as f:
    json.dump(metrics, f)

print("✅ Metrics saved to metrics_checkpoint.json")


✅ Metrics saved to metrics_checkpoint.json


In [29]:
good_question_rate = (good_question_count / communication_count * 100) if communication_count > 0 else 0
communication_rate = (communication_count / total_number_of_modified_questions * 100) if total_number_of_modified_questions > 0 else 0
pass_at_1_rate = (pass_at_1_count / total_number_of_questions * 100) if total_number_of_questions > 0 else 0
test_pass_rate = (test_pass_count / total_number_of_questions * 100) if total_number_of_questions > 0 else 0


In [30]:
print(f"Good question rate: {good_question_rate}")
print(f"Communication rate: {communication_rate}")
print(f"Pass at 1 rate: {pass_at_1_rate}")
print(f"Test pass rate: {test_pass_rate}")

Good question rate: 75.0
Communication rate: 4.150453955901426
Pass at 1 rate: 1.7112299465240641
Test pass rate: 3.2085561497326207


In [37]:
print(total_number_of_questions_new)

33


In [41]:
print(f"Good question rate: {(good_question_count/communication_count) * 100}")
print(f"Communication rate: {(communication_count/total_number_of_questions_new) * 100}")
print(f"Pass at 1 rate: {(pass_at_1_count/total_number_of_questions_new) * 100}")
print(f"Test pass rate: {(test_pass_count/total_number_of_questions_new) * 100}")

Good question rate: 75.0
Communication rate: 96.96969696969697
Pass at 1 rate: 48.484848484848484
Test pass rate: 90.9090909090909
